In [1]:
import pandas as pd
import numpy as np

In [9]:
result_ser = pd.read_csv('./mil classifier/reselts.txt',header = None)
result_ser.columns = ['learning rate','weight_decay','Tmax','best score','best auc','best avg auc']
result_star = pd.read_csv('./mil classifier/reselts_star.txt',header = None)
result_star.columns = ['learning rate','weight_decay','Tmax','best score','best auc','best avg auc']

In [15]:
result = pd.concat([result_ser,result_star])

In [16]:
result

,learning rate,weight_decay,Tmax,best score,best auc,best avg auc
0,0.000100,1.900000e-05,50,0.839332,0.412500,1.014583
1,0.000100,1.000000e-05,200,0.968894,0.572917,1.206250
2,0.000100,2.800000e-05,200,0.840374,0.412500,1.018750
3,0.000100,2.800000e-05,50,0.839332,0.412500,1.014583
4,0.000100,1.900000e-05,100,0.840374,0.412500,1.016667
...,...,...,...,...,...,...
1195,0.000091,8.200000e-06,200,0.910453,0.535417,1.162500
1196,0.000064,9.100000e-07,200,0.917277,0.518750,1.116667
1197,0.000091,9.100000e-06,100,0.927694,0.545833,1.170833
1198,0.000091,9.100000e-06,50,0.957902,0.535417,1.191667
